In [2]:
import numpy as np
import elimination_ordering as eo
import time, pickle
from os import listdir
from os.path import isfile, join
import pandas as pd

### Testing schemes

#### Matrix Market

In [11]:
'''matrices pre-processing'''
mypath = "matrices/matrix/mm/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
#for all matrices, load:
for f in onlyfiles:
    G_mm = eo.load_matrix_market("matrices/matrix/mm/"+f)
    G_sym = eo.symmetrization(G_mm)
    G_bip = eo.bipartization(G_mm)

    #save all matrices to metis format:
    eo.adj_mat_to_metis_file(G_sym, "matrices/matrix/ndmetis_input/"+f+".sym.metisgraph")
    eo.adj_mat_to_metis_file(G_bip, "matrices/matrix/ndmetis_input/"+f+".bip.metisgraph")
    
    #run the metis using all the matrices elsewhere.....



['bcspwr10.mtx.gz', 'bcsstk18.mtx.gz', 'bcsstm12.mtx.gz', 'bcsstm27.mtx.gz', 'blckhole.mtx.gz', 'can_1072.mtx.gz', 'cavity05.mtx.gz', 'cry10000.mtx.gz', 'cry2500.mtx.gz', 'e40r5000.mtx.gz', 'rdb3200l.mtx.gz', 'sstmodel.mtx.gz', 'west2021.mtx.gz']


NameError: name 'adj_mat_to_metis_file' is not defined

In [12]:
'''testing main for matrix market'''
#for all matrices, reload:
mm_path = "matrices/matrix/mm/"
mmfiles = np.array([f for f in listdir(mm_path) if isfile(join(mm_path, f))])
#onlyfiles = onlyfiles[[0,2]] #only for testing subslices
print(mmfiles)
combinedfiles = np.concatenate((np.array([mmfiles]).T, np.array([mmfiles]).T), axis=1).flatten()
print(combinedfiles)

#requires iperm info to load the mm in order
iperm_path = "matrices/matrix/ndmetis_iperm/"
ipermfiles = np.array([f for f in listdir(iperm_path) if isfile(join(iperm_path, f))])
substring_idx = -21 #backward index to ".gz."
print(ipermfiles)

nv = []
ne = []
mat_types = [] #bipartite or not, 1/0
times = []
fills_eli = []
fills_metis = []
fills_eli_ratio = []
fills_metis_ratio = []
eli_metis_ratios = []
#mat_metadata = []
es = [] #list of e from eli
for f in ipermfiles:
    G_mm = eo.load_matrix_market(mm_path+f[:substring_idx])
    G = None
    if ".sym." in f: #even fo symmetric, otherwise bipartite
        G = eo.symmetrization(G_mm); mat_types.append(False)
    elif ".bip." in f:
        G = eo.bipartization(G_mm); mat_types.append(True)
    #mat_metadata.append(metadata) #matrix metadata
    vertices = G.shape[0]
    edges = np.sum(G[np.triu_indices(G.shape[0], 1)]) #sum of upper triangular
    nv.append(vertices); ne.append(edges)
    #initialize: MUST always be in global scope:
    G1 = np.copy(G); G2 = np.copy(G)
    e, w, first_zero, last_zero, merge_forest, deleted = eo.initialize(G)

    #elimination ordering:
    start = time.time()
    eo.elimination_ordering(G)
    fill_eli, _ = eliminate(G1, e)
    fills_eli.append(fill_eli)
    es.append(e)
    print("eli",fill_eli,e)
    #print("sorted e", sorted(e))
    end = time.time()
    elapsed = end-start
    times.append(float('{0:.3g}'.format(elapsed)))
    print("time elapsed for elimination ordering: ",elapsed,"s")
    fills_eli_ratio.append(float('{0:.3g}'.format(float(fill_eli/edges))))

    #metis:
    metis_order = eo.iperm_to_orderlist(iperm_path+f)
    fill_metis, _ = eo.eliminate(G2, metis_order)
    fills_metis.append(fill_metis)
    print("metis", fill_metis, metis_order)
    fills_metis_ratio.append(float('{0:.3g}'.format(float(fill_metis/edges))))
    #print("sorted e", sorted(metis_order))
    #fills ratio between eli and metis:
    if (fill_eli != 0) and (fill_metis != 0): 
        eli_metis_ratios.append(float('{0:.3g}'.format(float(fill_eli/fill_metis)))) 
    else:
        eli_metis_ratios.append(0)
    
    '''print("mm",combinedfiles)
    #print("metadata",mat_metadata)
    print("nv",nv)
    print("ne",ne)
    print("mat_types",mat_types)
    print("eli",fills_eli)
    print("eli fills/sum(triu)",fills_eli_ratio)
    print("eli_times",times)
    print("metis",fills_metis)
    print("metis fills/sum(triu)",fills_metis_ratio)
    print("eli/metis ratio", eli_metis_ratios)'''

    #store results in file
    data = {
        "mm": combinedfiles,
        #"metadata": mat_metadata,
        "nv": nv,
        "ne": ne,
        "bipartite": mat_types,
        "eli_fills": fills_eli,
        "eli_ratio": fills_eli_ratio,
        "eli_times": times,
        "metis_fills": fills_metis,
        "metis_ratio": fills_metis_ratio,
        "eli/metis": eli_metis_ratios,
        "e_order":es
    }
    with open('matrices/matrix/matrix_data.p', 'wb') as fp:
        pickle.dump(data, fp)    
    with open('matrices/matrix/matrix_data.p', 'rb') as fp:
        data = pickle.load(fp)
        print("data = ",data)

'''#check ordering result:
e = np.array(e)
e_sort = np.sort(e[e != -1])
print(e_sort)
'''

['bcspwr10.mtx.gz' 'bcsstk18.mtx.gz' 'bcsstm12.mtx.gz' 'bcsstm27.mtx.gz'
 'blckhole.mtx.gz' 'can_1072.mtx.gz' 'cavity05.mtx.gz' 'cry10000.mtx.gz'
 'cry2500.mtx.gz' 'e40r5000.mtx.gz' 'rdb3200l.mtx.gz' 'sstmodel.mtx.gz'
 'west2021.mtx.gz']
['bcspwr10.mtx.gz' 'bcspwr10.mtx.gz' 'bcsstk18.mtx.gz' 'bcsstk18.mtx.gz'
 'bcsstm12.mtx.gz' 'bcsstm12.mtx.gz' 'bcsstm27.mtx.gz' 'bcsstm27.mtx.gz'
 'blckhole.mtx.gz' 'blckhole.mtx.gz' 'can_1072.mtx.gz' 'can_1072.mtx.gz'
 'cavity05.mtx.gz' 'cavity05.mtx.gz' 'cry10000.mtx.gz' 'cry10000.mtx.gz'
 'cry2500.mtx.gz' 'cry2500.mtx.gz' 'e40r5000.mtx.gz' 'e40r5000.mtx.gz'
 'rdb3200l.mtx.gz' 'rdb3200l.mtx.gz' 'sstmodel.mtx.gz' 'sstmodel.mtx.gz'
 'west2021.mtx.gz' 'west2021.mtx.gz']
['bcspwr10.mtx.gz.bip.metisgraph.iperm'
 'bcspwr10.mtx.gz.sym.metisgraph.iperm'
 'bcsstk18.mtx.gz.bip.metisgraph.iperm'
 'bcsstk18.mtx.gz.sym.metisgraph.iperm'
 'bcsstm12.mtx.gz.bip.metisgraph.iperm'
 'bcsstm12.mtx.gz.sym.metisgraph.iperm'
 'bcsstm27.mtx.gz.bip.metisgraph.iperm'
 'bcsstm

NameError: name 'deleted' is not defined

In [168]:
'''Table visualization'''
#pd.options.display.float_format = '{:,.2f}'.format
#mat_data_name = "matrix_data.p"
with open('data.p', 'rb') as fp:
    data = pickle.load(fp)
    #print("data = ",data)
    #print(len(data["mm"]), len(data["nv"]), len(data["eli/metis"]))
df = pd.DataFrame.from_dict(data)


df = df.drop(['e_order'], axis=1)
#df.style.set_precision(2)
df.style.format({
    'eli_ratio': '{:,.2f}'.format,
    'eli_times': '{:,.2f}'.format,
    'metis_ratio':'{:,.2f}'.format,
    'eli/metis': '{:,.2f}'.format
    #'B': '{:,.3f}'.format,
})



data =  {'mm': array(['bcsstk01.mtx.gz', 'bcsstk01.mtx.gz', 'bcsstm13.mtx.gz',
       'bcsstm13.mtx.gz', 'ck104.mtx.gz', 'ck104.mtx.gz',
       'gr_30_30.mtx.gz', 'gr_30_30.mtx.gz', 'mcca.mtx.gz', 'mcca.mtx.gz',
       'mcfe.mtx.gz', 'mcfe.mtx.gz', 'nnc1374.mtx.gz', 'nnc1374.mtx.gz',
       'will57.mtx.gz', 'will57.mtx.gz', 'zenios.mtx.gz', 'zenios.mtx.gz'],
      dtype='<U15'), 'nv': [96, 48, 4006, 2003, 208, 104, 1800, 900, 360, 180, 1530, 765, 2748, 1374, 114, 57, 5746, 2873], 'ne': [400.0, 176, 777.0, 62, 10.0, 4, 7744.0, 3422, 2648.0, 1680, 24344.0, 15352, 4286.0, 2248, 281.0, 127, 4.0, 2], 'bipartite': [True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False], 'eli_fills': [1769, 419, 62, 0, 2, 0, 367747, 23229, 5788, 964, 278468, 67826, 18348, 7891, 497, 24, 0, 0], 'eli_ratio': [4.42, 2.38, 0.0798, 0.0, 0.2, 0.0, 47.5, 6.79, 2.19, 0.574, 11.4, 4.42, 4.28, 3.51, 1.77, 0.189, 0.0, 0.0], 'eli_times': [0.797, 0.248, 241.0, 

,mm,nv,ne,bipartite,eli_fills,eli_ratio,eli_times,metis_fills,metis_ratio,eli/metis
0,bcsstk01.mtx.gz,96,400,True,1769,4.42,0.80,1173,2.93,1.51
1,bcsstk01.mtx.gz,48,176,False,419,2.38,0.25,265,1.51,1.58
2,bcsstm13.mtx.gz,4006,777,True,62,0.08,241.00,62,0.08,1.00
3,bcsstm13.mtx.gz,2003,62,False,0,0.00,37.40,0,0.00,0.00
4,ck104.mtx.gz,208,10,True,2,0.20,0.32,2,0.20,1.00
5,ck104.mtx.gz,104,4,False,0,0.00,0.08,0,0.00,0.00
6,gr_30_30.mtx.gz,1800,7744,True,367747,47.50,"1,930.00",42390,5.47,8.68
7,gr_30_30.mtx.gz,900,3422,False,23229,6.79,7.97,13410,3.92,1.73
8,mcca.mtx.gz,360,2648,True,5788,2.19,14.90,3462,1.31,1.67
9,mcca.mtx.gz,180,1680,False,964,0.57,1.73,334,0.20,2.89


#### Grids

In [149]:
'''Grid testing'''
'''save the grids to metis files first'''
'''p=q:'''
ndpath = "matrices/grid/ndmetis_input/"
gridpath = "matrices/grid/grids/"
ps = [2,5,10,15,20]
qs = []
for p in ps:
    #grid = grid_generator(0,0,k,p_dep=1,q_dep=1)
    q = p**2
    qs.append(q)
    grid = eo.grid_generator(p,q,0)
    print(grid.shape)
    #fname="p_q_"+str(k) #"p_q_" or "p_qsqr_" or others
    fname = str(p)+"_"+str(q)
    '''with open(gridpath+fname+".grid", 'wb') as fp:
        pickle.dump(grid, fp)'''
    eo.adj_mat_to_metis_file(grid, ndpath+fname+".grid.metisgraph")
#save grid sizes (p,q):
data = {"p":ps, "q":qs}
with open(gridpath+str(len(ps))+"_grids_sizes.info", 'wb') as fp:
    pickle.dump(data, fp)

p,q,k 2 4 0
(8, 8)
writing matrices/grid/ndmetis_input/2_4.grid.metisgraph done!
p,q,k 5 25 0
(125, 125)
writing matrices/grid/ndmetis_input/5_25.grid.metisgraph done!
p,q,k 10 100 0
(1000, 1000)
writing matrices/grid/ndmetis_input/10_100.grid.metisgraph done!
p,q,k 15 225 0
(3375, 3375)
writing matrices/grid/ndmetis_input/15_225.grid.metisgraph done!
p,q,k 20 400 0
(8000, 8000)
writing matrices/grid/ndmetis_input/20_400.grid.metisgraph done!


In [31]:
'''do elimination ordering and metis on each of the grids'''
gridpath = "matrices/grid/grids/"
ipermpath = "matrices/grid/ndmetis_iperm/"
files = np.array([f for f in listdir(gridpath) if isfile(join(gridpath, f))])
print(files)
nv = []
ne = []
#mat_types = [] #bipartite or not, 1/0
times = []
fills_eli = []
fills_metis = []
fills_eli_ratio = []
fills_metis_ratio = []
eli_metis_ratios = []
#mat_metadata = []
es = [] #list of e from eli
for file in files:
    grid = None
    with open(gridpath+file, 'rb') as fp:
        grid = pickle.load(fp)
    print("grid_shape",grid.shape)
    vertices = grid.shape[0]
    edges = np.sum(grid[np.triu_indices(grid.shape[0], 1)]) #sum of upper triangular
    nv.append(vertices); ne.append(edges)
    
    #elimination ordering:
    start = time.time()
    e, w, first_zero, last_zero, merge_forest, deleted = initialize(grid)
    elimination_ordering(grid)
    end = time.time()
    elapsed = end-start
    times.append(elapsed)
    print("time elapsed for elimination ordering: ",elapsed,"s")
    #grid is deleted above, so reload here:
    with open(gridpath+file, 'rb') as fp:
        grid = pickle.load(fp)
    fill_eli, _ = eliminate(grid, e)
    fills_eli.append(fill_eli)
    es.append(e)
    print("eli",fill_eli,e)
    fills_eli_ratio.append(float(fill_eli/edges))
    
    #metis:
    with open(gridpath+file, 'rb') as fp:
        grid = pickle.load(fp)
    metis_order = iperm_to_orderlist(ipermpath+file+'.metisgraph.iperm')
    fill_metis, _ = eliminate(grid, metis_order)
    fills_metis.append(fill_metis)
    print("metis", fill_metis, metis_order)
    fills_metis_ratio.append(float(fill_metis/edges))
    if (fill_eli != 0) and (fill_metis != 0): 
        eli_metis_ratios.append(float(fill_eli/fill_metis))
    else:
        eli_metis_ratios.append(0)

    #store results in file, within loop, so it will always store something even when stopped earlier
    data = {
        "grids": files,
        "nv": nv,
        "ne": ne,
        "eli_fills": fills_eli,
        "eli_ratio": fills_eli_ratio,
        "eli_times": times,
        "metis_fills": fills_metis,
        "metis_ratio": fills_metis_ratio,
        "eli/metis": eli_metis_ratios,
        "e_order":es
    }
    with open('matrices/grid/grid_data.p', 'wb') as fp:
        pickle.dump(data, fp)    
    with open('matrices/grid/grid_data.p', 'rb') as fp:
        data = pickle.load(fp)
        print(data)

['p_q_1.grid' 'p_q_2.grid' 'p_q_3.grid' 'p_q_4.grid' 'p_q_5.grid'
 'p_q_6.grid' 'p_q_7.grid']
grid_shape (4, 4)
time elapsed for elimination ordering:  0.001001119613647461 s
eli 1 [0 3 2 1]
metis 1 [3 0 2 1]
{'grids': array(['p_q_1.grid', 'p_q_2.grid', 'p_q_3.grid', 'p_q_4.grid',
       'p_q_5.grid', 'p_q_6.grid', 'p_q_7.grid'], dtype='<U10'), 'nv': [4], 'ne': [4.0], 'eli_fills': [1], 'eli_ratio': [0.25], 'eli_times': [0.001001119613647461], 'metis_fills': [1], 'metis_ratio': [0.25], 'eli/metis': [1.0], 'e_order': [array([0, 3, 2, 1])]}
grid_shape (16, 16)
stage iteration: 0
stage iteration: 1
time elapsed for elimination ordering:  0.011010408401489258 s
eli 21 [ 0  3 12 15  1  4  8 13  6  7  2 14 11 10  9  5]
metis 20 [ 3 15 10  7 14 11 12  8  5  0  1  4 13  9  6  2]
{'grids': array(['p_q_1.grid', 'p_q_2.grid', 'p_q_3.grid', 'p_q_4.grid',
       'p_q_5.grid', 'p_q_6.grid', 'p_q_7.grid'], dtype='<U10'), 'nv': [4, 16], 'ne': [4.0, 24.0], 'eli_fills': [1, 21], 'eli_ratio': [0.25, 0.875

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in double_scalars


stage iteration: 2
stage iteration: 3
stage iteration: 4
time elapsed for elimination ordering:  0.26824307441711426 s
eli 328 [ 0  7 56 63  5  6 14 20 23 29 57 59 48 30 31 36 39 45 22  3 12 32 34 35
 41 47 49 50 62 27 40 46 55 54 61 53 52 60 51 24  1  8 16 25 10 11  2 26
 19 18 17  9 33  4 42 43 44 37 38 58 15 28 21 13]
metis 220 [59 56 52 35 18  1 40 24  8 44  0 58 26 49 42 33  9 51 17 57 16 50 25 43
 32 48 34 41 61 63 37 28 11  7 54 47 31  5  3 62 20 15 22 55  4 46 38 13
 39 12 30  6 23 14 21 29  2 53 60 45 10 19 27 36]
{'grids': array(['p_q_1.grid', 'p_q_2.grid', 'p_q_3.grid', 'p_q_4.grid',
       'p_q_5.grid', 'p_q_6.grid', 'p_q_7.grid'], dtype='<U10'), 'nv': [4, 16, 64], 'ne': [4.0, 24.0, 112.0], 'eli_fills': [1, 21, 328], 'eli_ratio': [0.25, 0.875, 2.9285714285714284], 'eli_times': [0.001001119613647461, 0.011010408401489258, 0.26824307441711426], 'metis_fills': [1, 20, 220], 'metis_ratio': [0.25, 0.8333333333333334, 1.9642857142857142], 'eli/metis': [1.0, 1.05, 1.49090909090909

grid_shape (4096, 4096)
stage iteration: 0
stage iteration: 1
stage iteration: 2
stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7
stage iteration: 8
stage iteration: 9
stage iteration: 10
stage iteration: 11
stage iteration: 12
stage iteration: 13
stage iteration: 14
stage iteration: 15
stage iteration: 16
stage iteration: 17
stage iteration: 18
stage iteration: 19
stage iteration: 20
stage iteration: 21
stage iteration: 22
stage iteration: 23
stage iteration: 24
stage iteration: 25
stage iteration: 26
stage iteration: 27
stage iteration: 28
stage iteration: 29
stage iteration: 30
stage iteration: 31
stage iteration: 32
stage iteration: 33
stage iteration: 34
stage iteration: 35
stage iteration: 36
stage iteration: 37
stage iteration: 38
stage iteration: 39
stage iteration: 40
stage iteration: 41
stage iteration: 42
stage iteration: 43
stage iteration: 44
stage iteration: 45
stage iteration: 46
stage iteration: 47
stage iteration: 48
stage 

KeyboardInterrupt: 

In [159]:
'''All statistics for grids (+jointree)'''
'''do elimination ordering and metis on each of the grids'''
#gridpath = "matrices/grid/grids/"
ipermpath = "matrices/grid/ndmetis_iperm/"
files = np.array([f for f in listdir(ipermpath) if isfile(join(ipermpath, f))])
print(files)

#data containers:
nv = []
ne = []
#mat_types = [] #bipartite or not, 1/0
times = []
fills_eli = []
fills_metis = []
fills_eli_ratio = []
fills_metis_ratio = []
eli_metis_ratios = []
es = [] #list of e from eli
#jointree data:
max_C_eli = []
max_K_eli = []
max_C_metis = []
max_K_metis = []
#grids specific data:
ps = []; qs = []
fnames = []

for file in files:
    grid = None
    str_pq = file.split('.')[0]; fnames.append(str_pq+".grid")
    p,q = str_pq.split('_'); p,q = (int(p), int(q)); ps.append(p); qs.append(q)
    #rather than loading grids from disk, it'll be more space efficient if we just re-generate the grids in memory
    grid = grid_generator(p,q,0)
    print("grid_shape",grid.shape)
    vertices = grid.shape[0]
    edges = np.sum(grid[np.triu_indices(grid.shape[0], 1)]) #sum of upper triangular
    nv.append(vertices); ne.append(edges)
    #elimination ordering:
    start = time.time()
    e, w, first_zero, last_zero, merge_forest, deleted = initialize(grid)
    es.append(e)
    elimination_ordering(grid)
    end = time.time()
    elapsed = end-start
    times.append(elapsed)
    print("time elapsed for elimination ordering: ",elapsed,"s")
    #grid is deleted above, so reload here:
    grid = grid_generator(p,q,0)
    #eliminate + jointree generation:
    fill_eli, _, C_vs, sep_idxs = eliminate(grid, e,join_tree=True); print("eli||generating jointree for ",str_pq,"completed!")
    _, _, max_C, max_K = absorption(e, C_vs, sep_idxs); print("eli||absorption for",str_pq,"completed!")
    fills_eli.append(fill_eli)
    max_C_eli.append(max_C); max_K_eli.append(max_K); print("eli||max_C, max_K:",max_C, max_K)
    print("eli||fills",fill_eli)
    fills_eli_ratio.append(float(fill_eli/edges))
    print()
    #metis:
    grid = grid_generator(p,q,0)
    metis_order = iperm_to_orderlist(ipermpath+file)
    fill_metis, _, C_vs, sep_idxs = eliminate(grid, metis_order, join_tree=True)
    fills_metis.append(fill_metis)
    print("metis||generating jointree for ",str_pq,"completed!")    
    _, _, max_C, max_K = absorption(e, C_vs, sep_idxs)
    max_C_metis.append(max_C); max_K_metis.append(max_K); print("metis||max_C, max_K:",max_C, max_K)
    print("metis||absorption for",str_pq,"completed!")
    print("metis", fill_metis)
    fills_metis_ratio.append(float(fill_metis/edges))
    print()
    if (fill_eli != 0) and (fill_metis != 0): 
        eli_metis_ratios.append(float(fill_eli/fill_metis))
    else:
        eli_metis_ratios.append(0)

    #store results in file, within loop, so it will always store something even when stopped earlier
    data = {
        "grids": fnames,
        "nv": nv,
        "ne": ne,
        "eli_fills": fills_eli,
        "eli_ratio": fills_eli_ratio,
        "eli_times": times,
        "metis_fills": fills_metis,
        "metis_ratio": fills_metis_ratio,
        "eli/metis": eli_metis_ratios,
        "e_order":es,
        #jointree data:
        "max_C_eli": max_C_eli,
        "max_K_eli": max_K_eli,
        "max_C_metis": max_C_metis,
        "max_K_metis":max_K_metis
    }
    with open('matrices/grid/grid_data.p', 'wb') as fp:
        pickle.dump(data, fp)    
    with open('matrices/grid/grid_data.p', 'rb') as fp:
        data = pickle.load(fp)
        print(data)

['10_100.grid.metisgraph.iperm' '15_225.grid.metisgraph.iperm'
 '20_400.grid.metisgraph.iperm' '2_4.grid.metisgraph.iperm'
 '5_25.grid.metisgraph.iperm']
p,q,k 10 100 0
grid_shape (1000, 1000)
stage iteration: 0
stage iteration: 1
stage iteration: 2
stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7
stage iteration: 8
stage iteration: 9
stage iteration: 10
stage iteration: 11
stage iteration: 12
stage iteration: 13
stage iteration: 14
stage iteration: 15
stage iteration: 16
stage iteration: 17
stage iteration: 18
stage iteration: 19
stage iteration: 20
stage iteration: 21
stage iteration: 22
stage iteration: 23
stage iteration: 24
stage iteration: 25
stage iteration: 26
stage iteration: 27
stage iteration: 28
stage iteration: 29
stage iteration: 30
stage iteration: 31
stage iteration: 32
stage iteration: 33
stage iteration: 34
stage iteration: 35
stage iteration: 36
stage iteration: 37
stage iteration: 38
stage iteration: 39
stage iteration: 

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in double_scalars


eli||generating jointree for  10_100 completed!
eli||absorption for 10_100 completed!
eli||max_C, max_K: 296 294
eli||fills 57845

p,q,k 10 100 0
metis||generating jointree for  10_100 completed!
metis||max_C, max_K: 33 24
metis||absorption for 10_100 completed!
metis 7264

{'grids': ['10_100.grid'], 'nv': [1000], 'ne': [1890.0], 'eli_fills': [57845], 'eli_ratio': [30.605820105820104], 'eli_times': [868.854127407074], 'metis_fills': [7264], 'metis_ratio': [3.8433862433862434], 'eli/metis': [7.963243392070485], 'e_order': [array([  0,  99, 900, 999,   7,   8,  10, 108, 206, 307, 901, 903, 800,
        96,  98, 197, 198, 296, 299, 397, 989, 991,   6,  11, 106, 110,
       209, 305, 310, 403, 409, 411, 504, 508, 510, 607, 609, 708,   5,
       105, 210, 309,   4,   2, 201, 300, 302, 100, 101, 200,   1, 303,
       401, 501, 600, 602, 400, 500, 398, 399, 496, 499, 595, 696, 992,
        95,   3, 103, 104, 203, 204, 304, 405, 701, 801, 802, 603, 406,
       308, 408, 506, 407, 507, 605, 706

stage iteration: 101
stage iteration: 102
stage iteration: 103
stage iteration: 104
stage iteration: 105
stage iteration: 106
stage iteration: 107
stage iteration: 108
stage iteration: 109
stage iteration: 110
stage iteration: 111
stage iteration: 112
stage iteration: 113
stage iteration: 114
stage iteration: 115
stage iteration: 116
stage iteration: 117
stage iteration: 118
stage iteration: 119
stage iteration: 120
stage iteration: 121
stage iteration: 122
stage iteration: 123
stage iteration: 124
stage iteration: 125
stage iteration: 126
stage iteration: 127
stage iteration: 128
stage iteration: 129
stage iteration: 130
stage iteration: 131
stage iteration: 132
stage iteration: 133
stage iteration: 134
stage iteration: 135
stage iteration: 136
stage iteration: 137
stage iteration: 138
stage iteration: 139
stage iteration: 140
stage iteration: 141
stage iteration: 142
stage iteration: 143
stage iteration: 144
stage iteration: 145
stage iteration: 146
stage iteration: 147
stage iterati

stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7
stage iteration: 8
stage iteration: 9
stage iteration: 10
stage iteration: 11
stage iteration: 12
stage iteration: 13


KeyboardInterrupt: 

In [157]:
s = "2:3"
r,t = s.split(':'); r,t=(int(r),int(t))
print(r, t)

2 3


In [32]:
'''table vis'''
with open('matrices/grid/grid_data.p', 'rb') as fp:
    data = pickle.load(fp)
    print(data)
del data['grids']
df = pd.DataFrame.from_dict(data)
df = df.drop(['e_order'], axis=1)
#df.style.set_precision(2)
df.style.format({
    'eli_ratio': '{:,.2f}'.format,
    'eli_times': '{:,.2f}'.format,
    'metis_ratio':'{:,.2f}'.format,
    'eli/metis': '{:,.2f}'.format
    #'B': '{:,.3f}'.format,
})

{'grids': array(['p_q_1.grid', 'p_q_2.grid', 'p_q_3.grid', 'p_q_4.grid',
       'p_q_5.grid', 'p_q_6.grid', 'p_q_7.grid'], dtype='<U10'), 'nv': [4, 16, 64, 256, 1024, 4096], 'ne': [4.0, 24.0, 112.0, 480.0, 1984.0, 8064.0], 'eli_fills': [1, 21, 328, 3829, 56593, 728690], 'eli_ratio': [0.25, 0.875, 2.9285714285714284, 7.977083333333334, 28.52469758064516, 90.36334325396825], 'eli_times': [0.001001119613647461, 0.011010408401489258, 0.26824307441711426, 9.269427299499512, 573.7220952510834, 35339.19025707245], 'metis_fills': [1, 20, 220, 1571, 10614, 56614], 'metis_ratio': [0.25, 0.8333333333333334, 1.9642857142857142, 3.2729166666666667, 5.349798387096774, 7.020585317460317], 'eli/metis': [1.0, 1.05, 1.490909090909091, 2.4373010821133034, 5.33192010552101, 12.871197936906066], 'e_order': [array([0, 3, 2, 1]), array([ 0,  3, 12, 15,  1,  4,  8, 13,  6,  7,  2, 14, 11, 10,  9,  5]), array([ 0,  7, 56, 63,  5,  6, 14, 20, 23, 29, 57, 59, 48, 30, 31, 36, 39,
       45, 22,  3, 12, 32, 34, 35

,nv,ne,eli_fills,eli_ratio,eli_times,metis_fills,metis_ratio,eli/metis
0,4,4,1,0.25,0.00,1,0.25,1.00
1,16,24,21,0.88,0.01,20,0.83,1.05
2,64,112,328,2.93,0.27,220,1.96,1.49
3,256,480,3829,7.98,9.27,1571,3.27,2.44
4,1024,1984,56593,28.52,573.72,10614,5.35,5.33
5,4096,8064,728690,90.36,"35,339.19",56614,7.02,12.87


### Join Tree

In [107]:
#generate elimination ordering:
'''graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])'''

#example from 278:
graph = np.array([
    [0,0,1,0,1,0,0,0,0],
    [0,0,1,0,1,0,0,0,1],
    [1,1,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,1],
    [0,0,0,0,0,0,0,1,1],
    [0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,1,1,0,0],
    [0,1,0,0,1,1,0,1,0]
])

graph_elim = np.copy(graph)
graph_elim2 = np.copy(graph_elim)
e, w, first_zero, last_zero, merge_forest, deleted = initialize(graph) #must be on global scope
start = time.time()
elimination_ordering(graph)
fills, _ = eliminate(graph_elim, e)
print("eli",fills,e+1)
end = time.time()
print("time elapsed for elimination ordering: ",end - start,"s")


'''join tree testing'''
#e = [0,1,2,3,4,5,6,7,8]
_,_,C_vs,sep_idxs = eliminate(graph_elim2,e,join_tree=True)
absorption(e, C_vs, sep_idxs)

eli 2 [1 3 4 5 6 7 8 2 9]
time elapsed for elimination ordering:  0.0010008811950683594 s


C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in long_scalars


(array([3, 3, 2, 3, 3, 2]), array([2, 2, 1, 0, 1, 1]), 3, 2)

In [127]:
'''statistics of max C and max K from jointree'''
'''the block of codes below is for when the data is already calculated previously'''
path = 'matrices/archive/18.02.2021/matrix/'
#load previously computed data:
data_fname = path+'data.p'
with open(data_fname, 'rb') as fp:
    data = pickle.load(fp)
#print(data)

matrix_files = [path+"mm/"+mm for mm in data['mm']]
i = 0
max_C_eli = []
max_K_eli = []
max_C_metis = []
max_K_metis = []
for mfile in matrix_files:
    #create jointree using eli:
    G = load_matrix_market(mfile)
    if data["bipartite"][i] == True:
        G = bipartization(G)
    else:
        G = symmetrization(G)
    e = data['e_order'][i]
    _, _ , C_vs, sep_idxs = eliminate(G, e, join_tree=True)
    print("eli||generating jointree for ",mfile,"completed!")
    _, _, max_C, max_K = absorption(e, C_vs, sep_idxs)
    max_C_eli.append(max_C); max_K_eli.append(max_K)
    print("eli||absorption for",mfile,"completed!")
    print("eli||max_C, max_K:",max_C, max_K)
    print()
    #create jointree using metis:
    G = load_matrix_market(mfile)
    if data["bipartite"][i] == True:
        G = bipartization(G)
        e = iperm_to_orderlist(path+"ndmetis_iperm/"+data['mm'][i]+".bip.metisgraph.iperm")
    else:
        G = symmetrization(G)
        e = iperm_to_orderlist(path+"ndmetis_iperm/"+data['mm'][i]+".sym.metisgraph.iperm")
    _, _ , C_vs, sep_idxs = eliminate(G, e, join_tree=True)
    print("metis||generating jointree for ",mfile,"completed!")
    _, _, max_C, max_K = absorption(e, C_vs, sep_idxs)
    max_C_metis.append(max_C); max_K_metis.append(max_K)

    print("metis||absorption for",mfile,"completed!")
    print("metis||max_C, max_K:",max_C, max_K)
    print()
    
    #append all data and write to file
    data["max_C_eli"] = max_C_eli
    data["max_K_eli"] = max_K_eli
    data["max_C_metis"] = max_C_metis
    data["max_K_metis"] = max_K_metis
    with open(path+"new_matrix_data.jt.p", 'wb') as fp:
        pickle.dump(data, fp)    
    
    i+=1
    print("next iter! \n")

eli||generating jointree for  matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
eli||absorption for matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
eli||max_C, max_K: 55 51

metis||generating jointree for  matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
metis||absorption for matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
metis||max_C, max_K: 37 35

next iter! 

eli||generating jointree for  matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
eli||absorption for matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
eli||max_C, max_K: 25 24

metis||generating jointree for  matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
metis||absorption for matrices/archive/18.02.2021/matrix/mm/bcsstk01.mtx.gz completed!
metis||max_C, max_K: 18 15

next iter! 

eli||generating jointree for  matrices/archive/18.02.2021/matrix/mm/bcsstm13.mtx.gz completed!
eli||absorption for matrices/archive/18.02.

In [7]:
with open('matrices/archive/18.02.2021/matrix/new_matrix_data.jt.p', 'rb') as fp:
    data = pickle.load(fp)
    #print(data)
df = pd.DataFrame.from_dict(data)
df = df.drop(['e_order'], axis=1)
df.style.format({
    'eli_ratio': '{:,.2f}'.format,
    'eli_times': '{:,.2f}'.format,
    'metis_ratio':'{:,.2f}'.format,
    'eli/metis': '{:,.2f}'.format,
    'ne': '{:,.0f}'.format,
    'nv': '{:,.0f}'.format,
    'eli_fills': '{:,.0f}'.format,
    'metis_fills': '{:,.0f}'.format
    #'B': '{:,.3f}'.format,
})

,mm,nv,ne,bipartite,eli_fills,eli_ratio,eli_times,metis_fills,metis_ratio,eli/metis,max_C_eli,max_K_eli,max_C_metis,max_K_metis
0,bcsstk01.mtx.gz,96,400,True,"1,769",4.42,0.80,"1,173",2.93,1.51,55,51,37,35
1,bcsstk01.mtx.gz,48,176,False,419,2.38,0.25,265,1.51,1.58,25,24,18,15
2,bcsstm13.mtx.gz,"4,006",777,True,62,0.08,241.00,62,0.08,1.00,3,2,3,2
3,bcsstm13.mtx.gz,"2,003",62,False,0,0.00,37.40,0,0.00,0.00,2,0,2,0
4,ck104.mtx.gz,208,10,True,2,0.20,0.32,2,0.20,1.00,3,2,3,2
5,ck104.mtx.gz,104,4,False,0,0.00,0.08,0,0.00,0.00,2,0,2,0
6,gr_30_30.mtx.gz,"1,800","7,744",True,"367,747",47.50,"1,930.00","42,390",5.47,8.68,706,681,88,76
7,gr_30_30.mtx.gz,900,"3,422",False,"23,229",6.79,7.97,"13,410",3.92,1.73,33,32,56,39
8,mcca.mtx.gz,360,"2,648",True,"5,788",2.19,14.90,"3,462",1.31,1.67,70,67,52,44
9,mcca.mtx.gz,180,"1,680",False,964,0.57,1.73,334,0.20,2.89,51,48,45,28


In [136]:
'''for grids'''
path = 'matrices/archive/18.02.2021/grid/'
#load previously computed data:
data_fname = path+'grid_data.p'
with open(data_fname, 'rb') as fp:
    data = pickle.load(fp)
#temporary slicing because not all data are computed:
data["grids"] = data["grids"][:len(data["eli_fills"])]
#print(data)

grid_files = [path+"grids/"+grid for grid in data['grids']]
i = 0
max_C_eli = []
max_K_eli = []
max_C_metis = []
max_K_metis = []
for gfile in grid_files:
    #create jointree using eli:
    grid = None
    with open(gfile, 'rb') as fp:
        grid = pickle.load(fp)
    e = data['e_order'][i]
    _, _ , C_vs, sep_idxs = eliminate(grid, e, join_tree=True)
    print("eli||generating jointree for ",gfile,"completed!")
    _, _, max_C, max_K = absorption(e, C_vs, sep_idxs)
    max_C_eli.append(max_C); max_K_eli.append(max_K)
    print("eli||absorption for",gfile,"completed!")
    print("eli||max_C, max_K:",max_C, max_K)
    print()
    #create jointree using metis:
    with open(gfile, 'rb') as fp:
        grid = pickle.load(fp)
    e = iperm_to_orderlist(path+"ndmetis_iperm/"+data['grids'][i]+".metisgraph.iperm")
    _, _ , C_vs, sep_idxs = eliminate(grid, e, join_tree=True)
    print("metis||generating jointree for ",mfile,"completed!")
    _, _, max_C, max_K = absorption(e, C_vs, sep_idxs)
    max_C_metis.append(max_C); max_K_metis.append(max_K)

    print("metis||absorption for",gfile,"completed!")
    print("metis||max_C, max_K:",max_C, max_K)
    print()
    
    #append all data and write to file
    data["max_C_eli"] = max_C_eli
    data["max_K_eli"] = max_K_eli
    data["max_C_metis"] = max_C_metis
    data["max_K_metis"] = max_K_metis
    with open(path+"new_grid_data.jt.p", 'wb') as fp:
        pickle.dump(data, fp)
    
    i+=1
    print("next iter! \n")

eli||generating jointree for  matrices/archive/18.02.2021/grid/grids/p_q_1.grid completed!
eli||absorption for matrices/archive/18.02.2021/grid/grids/p_q_1.grid completed!
eli||max_C, max_K: 3 2

metis||generating jointree for  matrices/archive/18.02.2021/matrix/mm/zenios.mtx.gz completed!
metis||absorption for matrices/archive/18.02.2021/grid/grids/p_q_1.grid completed!
metis||max_C, max_K: 3 2

next iter! 

eli||generating jointree for  matrices/archive/18.02.2021/grid/grids/p_q_2.grid completed!
eli||absorption for matrices/archive/18.02.2021/grid/grids/p_q_2.grid completed!
eli||max_C, max_K: 6 4

metis||generating jointree for  matrices/archive/18.02.2021/matrix/mm/zenios.mtx.gz completed!
metis||absorption for matrices/archive/18.02.2021/grid/grids/p_q_2.grid completed!
metis||max_C, max_K: 5 4

next iter! 

eli||generating jointree for  matrices/archive/18.02.2021/grid/grids/p_q_3.grid completed!
eli||absorption for matrices/archive/18.02.2021/grid/grids/p_q_3.grid completed!
el

In [ ]:
'''All statistics for grids (+jointree)'''
'''do elimination ordering and metis on each of the grids'''
#gridpath = "matrices/grid/grids/"
ipermpath = "matrices/archive/18.02.2021/grid/ndmetis_iperm/"
files = np.array([f for f in listdir(ipermpath) if isfile(join(ipermpath, f))])
print(files)

#data containers:
nv = []
ne = []
#mat_types = [] #bipartite or not, 1/0
times = []
fills_eli = []
fills_metis = []
fills_eli_ratio = []
fills_metis_ratio = []
eli_metis_ratios = []
es = [] #list of e from eli
#jointree data:
max_C_eli = []
max_K_eli = []
max_C_metis = []
max_K_metis = []
#grids specific data:
ps = []; qs = []
fnames = []

for file in files:
    grid = None
    str_pq = file.split('.')[0]; fnames.append(str_pq+".grid")
    p,q = str_pq.split('_'); p,q = (int(p), int(q)); ps.append(p); qs.append(q)
    #rather than loading grids from disk, it'll be more space efficient if we just re-generate the grids in memory
    grid = eo.grid_generator(p,q,0)
    print("grid_shape",grid.shape)
    vertices = grid.shape[0]
    edges = np.sum(grid[np.triu_indices(grid.shape[0], 1)]) #sum of upper triangular
    nv.append(vertices); ne.append(edges)
    #elimination ordering:
    start = time.time()
    EO = eo.elimination_ordering_class(grid, visualization=False) #must be on global scope
    e = EO.elimination_ordering(grid)
    es.append(e)
    end = time.time()
    elapsed = end-start
    times.append(elapsed)
    print("time elapsed for elimination ordering: ",elapsed,"s")
    #grid is deleted above, so reload here:
    grid = eo.grid_generator(p,q,0)
    #eliminate + jointree generation:
    fill_eli, _, C_vs, sep_idxs = eo.eliminate(grid, e,join_tree=True); print("eli||generating jointree for ",str_pq,"completed!")
    _, _, max_C, max_K = eo.absorption(e, C_vs, sep_idxs); print("eli||absorption for",str_pq,"completed!")
    fills_eli.append(fill_eli)
    max_C_eli.append(max_C); max_K_eli.append(max_K); print("eli||max_C, max_K:",max_C, max_K)
    print("eli||fills",fill_eli)
    fills_eli_ratio.append(float(fill_eli/edges))
    print()
    #metis:
    grid = eo.grid_generator(p,q,0)
    metis_order = eo.iperm_to_orderlist(ipermpath+file)
    fill_metis, _, C_vs, sep_idxs = eo.eliminate(grid, metis_order, join_tree=True)
    fills_metis.append(fill_metis)
    print("metis||generating jointree for ",str_pq,"completed!")    
    _, _, max_C, max_K = eo.absorption(e, C_vs, sep_idxs)
    max_C_metis.append(max_C); max_K_metis.append(max_K); print("metis||max_C, max_K:",max_C, max_K)
    print("metis||absorption for",str_pq,"completed!")
    print("metis", fill_metis)
    fills_metis_ratio.append(float(fill_metis/edges))
    print()
    if (fill_eli != 0) and (fill_metis != 0): 
        eli_metis_ratios.append(float(fill_eli/fill_metis))
    else:
        eli_metis_ratios.append(0)

    #store results in file, within loop, so it will always store something even when stopped earlier
    data = {
        "grids": fnames,
        "nv": nv,
        "ne": ne,
        "eli_fills": fills_eli,
        "eli_ratio": fills_eli_ratio,
        "eli_times": times,
        "metis_fills": fills_metis,
        "metis_ratio": fills_metis_ratio,
        "eli/metis": eli_metis_ratios,
        "e_order":es,
        #jointree data:
        "max_C_eli": max_C_eli,
        "max_K_eli": max_K_eli,
        "max_C_metis": max_C_metis,
        "max_K_metis":max_K_metis
    }
    with open('matrices/archive/18.02.2021/grid/new_grid_data_25022021.jt.p', 'wb') as fp:
        pickle.dump(data, fp)    
    with open('matrices/archive/18.02.2021/grid/new_grid_data_25022021.jt.p', 'rb') as fp:
        data = pickle.load(fp)
        print(data)

['16_16.grid.metisgraph.iperm' '2_2.grid.metisgraph.iperm'
 '32_32.grid.metisgraph.iperm' '4_4.grid.metisgraph.iperm'
 '64_64.grid.metisgraph.iperm' '8_8.grid.metisgraph.iperm']
p,q,k 16 16 0
grid_shape (256, 256)
stage iteration: 0
stage iteration: 1
stage iteration: 2
stage iteration: 3
stage iteration: 4
time elapsed for elimination ordering:  5.550050258636475 s
p,q,k 16 16 0
eli||generating jointree for  16_16 completed!


C:\Users\beryl\Documents\Coding\Python\eli\elimination_ordering.py:83: RuntimeWarning: invalid value encountered in double_scalars
  mean_valency = np.sum(valencies)/n #get mean valency


eli||absorption for 16_16 completed!
eli||max_C, max_K: 28 18
eli||fills 1759

p,q,k 16 16 0
metis||generating jointree for  16_16 completed!
metis||max_C, max_K: 29 19
metis||absorption for 16_16 completed!
metis 1571

{'grids': ['16_16.grid'], 'nv': [256], 'ne': [480.0], 'eli_fills': [1759], 'eli_ratio': [3.6645833333333333], 'eli_times': [5.550050258636475], 'metis_fills': [1571], 'metis_ratio': [3.2729166666666667], 'eli/metis': [1.1196690006365373], 'e_order': [array([  0,  15, 240, 255,  12,  14,  28,  30,  31,  43,  45,  47,  58,
        60,  73,  75,  88,  90, 103, 105, 118, 120, 133, 135, 148, 150,
       163, 165, 178, 180, 193, 195, 208, 210, 225, 241, 242,  11,  63,
       192, 243,  10, 176, 160,   7,   9,  22,  24,  27,  26,  37,  39,
        52,  54,  67,  69,  82,  84,  96,  97,  99, 114, 128, 177, 161,
         6,  25,  42,  41,  80, 129, 144, 145, 162, 146,   5,  40,  57,
        56,  64, 130, 147, 131,   4,  55,  72,  71, 115, 132, 116,  70,
        87,  86, 100, 117

stage iteration: 0
stage iteration: 1
stage iteration: 2
stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7


In [15]:
with open('matrices/archive/18.02.2021/grid/new_grid_data.jt.p', 'rb') as fp:
    data = pickle.load(fp)
    #print(data)
df = pd.DataFrame.from_dict(data)
df = df.drop(['e_order'], axis=1)
df.style.format({
    'eli_ratio': '{:,.2f}'.format,
    'eli_times': '{:,.2f}'.format,
    'metis_ratio':'{:,.2f}'.format,
    'eli/metis': '{:,.2f}'.format,
    'ne': '{:,.0f}'.format,
    'nv': '{:,.0f}'.format,
    'eli_fills': '{:,.0f}'.format,
    'metis_fills': '{:,.0f}'.format
    #'B': '{:,.3f}'.format,
})

,grids,nv,ne,eli_fills,eli_ratio,eli_times,metis_fills,metis_ratio,eli/metis,max_C_eli,max_K_eli,max_C_metis,max_K_metis
0,p_q_1.grid,4,4,1,0.25,0.00,1,0.25,1.00,3,2,3,2
1,p_q_2.grid,16,24,21,0.88,0.01,20,0.83,1.05,6,4,5,4
2,p_q_3.grid,64,112,328,2.93,0.27,220,1.96,1.49,18,16,12,9
3,p_q_4.grid,256,480,"3,829",7.98,9.27,"1,571",3.27,2.44,74,57,29,19
4,p_q_5.grid,"1,024","1,984","56,593",28.52,573.72,"10,614",5.35,5.33,268,254,51,45
5,p_q_6.grid,"4,096","8,064","728,690",90.36,"35,339.19","56,614",7.02,12.87,1009,959,94,74


In [21]:
np.sqrt(256)

16.0

In [ ]:
'''print as ascii table'''
ps = [5,25,2,5]
qs = [5,25,4,25]
data = {
    "p": [],
    "q": [],
    "e_order": [],
    "place_loc": [],
    "round": []
       }

for h in range(len(ps)):
    grid = eo.grid_generator(ps[h], qs[h], 0)
    EO_obj = eo.elimination_ordering_class(grid, visualization=True)
    e = EO_obj.elimination_ordering(grid)
    print(e, EO_obj.place_loc, EO_obj.rounds_e)
    A = np.array([["0"*10]*qs[h]]*ps[h]) #length of string equals to 10 for each element of the matrix
    for i in range(e.shape[0]):
        order = e[i]
        x_idx = order%qs[h]
        y_idx = int(order/qs[h])
        sign = "+" if EO_obj.place_loc[i] == 0 else "-"
        A[y_idx][x_idx] = str(int(EO_obj.rounds_e[i]))+"_"+sign+str(i) #fill by "<round>_<sign of stage><elimination order>"
    
    #save calculation to file:
    data["p"].append(ps[h]); data["q"].append(qs[h]); data["e_order"].append(e); 
    data["place_loc"].append(EO_obj.place_loc); data["round"].append(EO_obj.rounds_e)
    with open("matrices/grid/grid_data_25022021.pr.p", 'wb') as fp:
        pickle.dump(data, fp)
    
    np.savetxt('documentation/grid_'+str(ps[h])+'_'+str(qs[h])+'.csv',A,fmt='%10s', delimiter=' ')